<a href="https://colab.research.google.com/github/DanielPreni/microsoft-graph-data-pull/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import msal

# Azure AD app details
CLIENT_ID =         # Replace with your app's Client ID
CLIENT_SECRET = ' # Replace with your Client Secret
TENANT_ID = ''        # Replace with your Tenant ID
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["https://graph.microsoft.com/.default"]

# Function to get an access token
def get_access_token():
    app = msal.ConfidentialClientApplication(
        CLIENT_ID, authority=AUTHORITY, client_credential=CLIENT_SECRET
    )
    result = app.acquire_token_for_client(scopes=SCOPE)
    if "access_token" in result:
        return result["access_token"]
    else:
        print("Error obtaining token:", result.get("error_description"))
        return None

# Test the function
token = get_access_token()
if token:
    print(f"Access token: {token}")


Access token: eyJ0eXAiOiJKV1QiLCJub25jZSI6InUtOTJXM0FqeHJmbGdTRDBJYnN4SmJOZ1RzZGlwbjE3MXlfWFFCQ29UU1EiLCJhbGciOiJSUzI1NiIsIng1dCI6Ik1jN2wzSXo5M2c3dXdnTmVFbW13X1dZR1BrbyIsImtpZCI6Ik1jN2wzSXo5M2c3dXdnTmVFbW13X1dZR1BrbyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC82ODIxZGQ1Zi0xZmMwLTRhYmEtYWYzOC0xY2VmMTg3MzA5NzkvIiwiaWF0IjoxNzI4MjU3MjU1LCJuYmYiOjE3MjgyNTcyNTUsImV4cCI6MTcyODI2MTE1NSwiYWlvIjoiazJCZ1lFaEpVdHBaMFZPeHQvQkYyTE10bXQ1eUFBPT0iLCJhcHBfZGlzcGxheW5hbWUiOiJPZmZpY2UgMzY1IGRhdGEgcHVsbCIsImFwcGlkIjoiNzUzN2U5N2MtY2Q4MS00OTA1LTk4MWYtMzY3OWFiZmE3M2FmIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNjgyMWRkNWYtMWZjMC00YWJhLWFmMzgtMWNlZjE4NzMwOTc5LyIsImlkdHlwIjoiYXBwIiwib2lkIjoiNmMyZjI3YTYtZTliYi00Y2M2LTg0MDQtYjUwYmExYjU2MmQ5IiwicmgiOiIwLkFSTUJYOTBoYU1BZnVrcXZPQnp2R0hNSmVRTUFBQUFBQUFBQXdBQUFBQUFBQUFBVEFRQS4iLCJyb2xlcyI6WyJEaXJlY3RvcnkuUmVhZC5BbGwiXSwic3ViIjoiNmMyZjI3YTYtZTliYi00Y2M2LTg0MDQtYjUwYmExYjU2MmQ5IiwidGVuYW50X3JlZ2lvbl9zY

In [4]:
!pip install requests openpyxl


In [ ]:
import requests
import openpyxl
from datetime import datetime

# API Endpoint
GRAPH_API_ENDPOINT = "https://graph.microsoft.com/v1.0"

# Headers for the request with the access token
def get_headers(token):
    return {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

# Function to get users data from Microsoft Graph
def get_users_data(token):
    headers = get_headers(token)
    users_data = []

    # Make the API call to get user data
    response = requests.get(f'{GRAPH_API_ENDPOINT}/users', headers=headers)

    if response.status_code == 200:
        users = response.json().get('value', [])

        # Debug: Print raw user data to inspect what's being returned
        for user in users:
            print(f"Raw User Data for {user.get('userPrincipalName')}:\n{user}\n")

            # Extract licenses (skuId or display name)
            licenses = [license.get('skuId') for license in (user.get('assignedLicenses') or [])]
            print(f"Licenses for {user.get('userPrincipalName')}: {licenses}\n")  # Debugging licenses

            # Ensure we're capturing all fields
            user_info = {
                'email': user.get('mail'),
                'username': user.get('userPrincipalName'),
                'licenses': licenses if licenses else ['No Licenses'],
                'date_created': user.get('createdDateTime', 'Not Available'),
                'last_login': get_last_login(token, user.get('id'))
            }
            users_data.append(user_info)
    else:
        print(f"Error fetching users data: {response.json()}")

    return users_data

# Function to get last login details (if available)
def get_last_login(token, user_id):
    headers = get_headers(token)
    response = requests.get(f"{GRAPH_API_ENDPOINT}/auditLogs/signIns?$filter=userId eq '{user_id}'", headers=headers)

    if response.status_code == 200:
        logins = response.json().get('value', [])

        # Debug: Print raw login data to inspect what's being returned
        print(f"Login Data for User {user_id}:\n{response.json()}\n")

        if logins:
            return logins[0].get('createdDateTime')  # Last login datetime
    return 'No Login Data'

# Function to write data to an Excel file
def write_to_excel(users_data):
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    # Add header row
    sheet.append(['Email Address', 'Username', 'Licenses Assigned', 'Date Created', 'Last Successful Login'])

    # Populate data
    for user in users_data:
        sheet.append([
            user['email'],
            user['username'],
            ', '.join(user['licenses']),
            user['date_created'],
            user['last_login']
        ])

    # Save the Excel file
    filename = f"Office365_Users_{datetime.now().strftime('%Y%m%d')}.xlsx"
    workbook.save(filename)
    print(f"Data has been written to {filename}")

# Main Execution: Fetch user data and write to Excel
if __name__ == '__main__':
    token = get_access_token()  # Assuming `get_access_token` is defined in your environment

    if token:
        users_data = get_users_data(token)
        if users_data:
            write_to_excel(users_data)
